In [10]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests

rawpath = '../raw'

In [11]:
df = pd.read_excel(f"{rawpath}/sru_test.xlsx",names=['nachname','vorname','titel','verlag','ort','jahr'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df

,nachname,vorname,titel,verlag,ort,jahr
0,Bateson,Maggie,Das @Puppenhaus,Ars-Ed.,München,1992
1,Lassert,Ursula,Alles über Papier,Auer,Donauwörth,1993
2,Weber-Biller,Bettina,Amir,Biller Weber,Will,1993
3,Meyer,Raimund,Dada,Kunsthalle der Hypo-Kulturstiftung,München,1993
4,Andresen,Ute,ABC und alles auf der Welt,Maier,Ravensburg,1993
5,Sauer,Karl-Ludwig,Das @Eindimensionale ist Nichts,Sauer,Buchholz,1993
6,Zauleck,Franz,Tannenbaum,Kinderbuchverl.,Berlin,1993
7,Sauer,Karl-Ludwig,Professorenstruktur,Verl. für das Künstlerbuch,Berlin,2001


In [12]:
base_url = 'https://services.dnb.de/sru/authorities?'
params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [13]:
def gnd_abfrage(row):
    params.update({'query': f'PER = "{row.nachname}, {row.vorname}" AND BBG=Tp*'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Authority'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None

            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [14]:
df['idnperson'] = df.apply(gnd_abfrage, axis=1)
df

,nachname,vorname,titel,verlag,ort,jahr,idnperson
0,Bateson,Maggie,Das @Puppenhaus,Ars-Ed.,München,1992,[]
1,Lassert,Ursula,Alles über Papier,Auer,Donauwörth,1993,[{'idn': '113693400'}]
2,Weber-Biller,Bettina,Amir,Biller Weber,Will,1993,[{'idn': '121607097'}]
3,Meyer,Raimund,Dada,Kunsthalle der Hypo-Kulturstiftung,München,1993,"[{'idn': '124842771'}, {'idn': '1151886777'}]"
4,Andresen,Ute,ABC und alles auf der Welt,Maier,Ravensburg,1993,[{'idn': '119522314'}]
5,Sauer,Karl-Ludwig,Das @Eindimensionale ist Nichts,Sauer,Buchholz,1993,[{'idn': '118605771'}]
6,Zauleck,Franz,Tannenbaum,Kinderbuchverl.,Berlin,1993,[{'idn': '132843935'}]
7,Sauer,Karl-Ludwig,Professorenstruktur,Verl. für das Künstlerbuch,Berlin,2001,[{'idn': '118605771'}]


In [22]:
def gnd_abfrage1(row):
    params.update({'query': f'PER = "{row.nachname}" AND TIT ="{row.titel}" AND JHR = "{row.jahr}" AND SIG = "dbsm*"'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")
    

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Bibliographic'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None

            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [23]:
df['idntitel'] = df.apply(gnd_abfrage1, axis=1)
df

,nachname,vorname,titel,verlag,ort,jahr,idnperson,idntitel
0,Bateson,Maggie,Das @Puppenhaus,Ars-Ed.,München,1992,[],[]
1,Lassert,Ursula,Alles über Papier,Auer,Donauwörth,1993,[{'idn': '113693400'}],[]
2,Weber-Biller,Bettina,Amir,Biller Weber,Will,1993,[{'idn': '121607097'}],[]
3,Meyer,Raimund,Dada,Kunsthalle der Hypo-Kulturstiftung,München,1993,"[{'idn': '124842771'}, {'idn': '1151886777'}]",[]
4,Andresen,Ute,ABC und alles auf der Welt,Maier,Ravensburg,1993,[{'idn': '119522314'}],[]
5,Sauer,Karl-Ludwig,Das @Eindimensionale ist Nichts,Sauer,Buchholz,1993,[{'idn': '118605771'}],[]
6,Zauleck,Franz,Tannenbaum,Kinderbuchverl.,Berlin,1993,[{'idn': '132843935'}],[]
7,Sauer,Karl-Ludwig,Professorenstruktur,Verl. für das Künstlerbuch,Berlin,2001,[{'idn': '118605771'}],[]
